
<h3 style="text-align: center;"><b>Школа глубокого обучения ФПМИ МФТИ</b></h3>
<h3 style="text-align: center;"><b>Индивидуальный проект. Reinforcement Learning. Deep Q Network</b></h3>


<b>Небольшое введение</b>

Определение из Википедии

<i>Обучение с подкреплением (англ. reinforcement learning) — один из способов машинного обучения, в ходе которого испытуемая система (агент) обучается, взаимодействуя с некоторой средой. С точки зрения кибернетики, является одним из видов кибернетического эксперимента. Откликом среды (а не специальной системы управления подкреплением, как это происходит в обучении с учителем) на принятые решения являются сигналы подкрепления, поэтому такое обучение является частным случаем обучения с учителем, но учителем является среда или её модель. Также нужно иметь в виду, что некоторые правила подкрепления базируются на неявных учителях, например, в случае искусственной нейронной среды, на одновременной активности формальных нейронов, из-за чего их можно отнести к обучению без учителя.</i>


<center><img src="https://econophysica.ru/upload/medialibrary/2e1/2e1784d383ec807d567db793d20f31f2.png"></center>

<b>В качестве цели данного проекта я решил взять обучение сверточной нейросети для игры в "Atari Breakout" с использованием алгоритма DQN (Deep Q Network). </b>


<center><img src="https://openai.com/content/images/2017/06/spaceinvadersbehavior.gif"></center>

Но для начала займемся настройкой среды. Библиотека OpenAI Gym предоставляет доступную из коробки среду для игр Atari и многих других, подробнее можно прочитать <a href="https://www.gymlibrary.ml/environments/atari/breakout/">в документации</a>. Однако я в своей работе другую библиотеку <a href="https://stable-baselines3.readthedocs.io/en/master/">Stable Baselines3</a> где помимо среды доступны некоторые обертки над ней, которые используются в <a href="https://arxiv.org/pdf/1312.5602.pdf">статье от DeepMind</a>. 

In [ ]:
from stable_baselines3 import DQN
import torch
from stable_baselines3.common.vec_env import SubprocVecEnv, DummyVecEnv
from stable_baselines3.common.env_util import make_atari_env
from stable_baselines3.common.vec_env import VecFrameStack, VecTransposeImage, VecNormalize
import datetime
from stable_baselines3.common.logger import configure
from typing import Callable

# Метод make_atari_env создает среду Breakout с Observation Space 210x160x3, внутри он делает следующие операции:
    # NoopReset: obtain initial state by taking random number of no-ops on reset.
    # Frame skipping: 4 by default
    # Max-pooling: most recent two observations
    # Termination signal when a life is lost.
    # Resize to a square image: 84x84 by default
    # Grayscale observation
    # Clip reward to {-1, 0, 1}
# Так же можно заметить, что он создает n_envs сред и обучение будет происходить параллельно в несколько потоков
envs = make_atari_env("ALE/Breakout-v5", n_envs=4, vec_env_cls=DummyVecEnv)
# Тут мы уже можем видить изображения размером 84х84х1
print(envs.observation_space.shape)
# Трансформируем из формата H, W, C в формат C, H, W
envs = VecTransposeImage(envs)
print(envs.observation_space.shape)
# Собираем несколько наблюдений вместе
envs = VecFrameStack(envs, n_stack=4)
print(envs.observation_space.shape)
# Нормализуем наблюдения, а также награду
envs = VecNormalize(envs)
print(envs.observation_space.shape)
# Таким образом мы имеем доступную из коробки среду, которая полностью удовлетворяет нашему формату и
# не требует дальнейших изменений. Т.е. значения Observations и Rewards будут представлены в таком формате,
# в котором мы бы хотели их видеть.

А теперь про сам алгоритм. Наша цель натренировать модель таким образом, чтобы суммарная дисконтированная награда была максимальной. 
$$R_{t0} = \sum_{t=t0}^\infty{\gamma}^{t-t0}{r_t}$$


Основная идея Q-learning состоит в том, что если мы имеем функцию 
$$ {Q^*}:{State}\times{Action} \rightarrow \mathbb{R} $$
которая может сказать нам, какой будет награда, если мы примем то или иное решение в текущем состоянии, мы можем легко написать функцию политики(Policy function), которая максимизирует нашу награду

$$\pi(s) = \{\operatorname{\argmax}}  Q^*(s,a)$$

Однако мы не знаем всего о среде, поэтому у нас нет доступа к $ Q^* $
Но, поскольку нейронные сети являются универсальными аппроксиматорами функций, мы можем просто создать их и обучить, чтобы они восстанавливали функцию $ Q^* $.

Для нашего обучения мы будем использовать тот факт, что каждая функция $Q$ для некоторой политики подчиняется уравнению Беллмана:
$$Q^\pi (s,a) = r + \gamma Q^\pi(s', \pi(s'))$$

Разница правой и левой частью уравнения называется temporal difference error $\delta$
$$\delta = Q(s,a) - (r+\gamma\max_a Q(s',a))$$

Чтобы минимизировать эту ошибку, мы будем использовать Huber loss. Huber loss действует как MSE, когда ошибка мала, но как MAE, когда ошибка велика — это делает ее более устойчивой к выбросам, когда оценки $Q$ очень зашумлены. Мы рассчитываем это по батчу "опытов" (transitions) $B$, взятых из replay memory:

$$\mathcal{L} = \frac{1}{|B|} \sum_{(s,a,s',r) \in B} \mathcal{L}(\delta)$$


$$\mathcal{L}(\delta) = \begin{cases}
    \frac{1}{2}(\delta)^2 & for |\delta| \le 1, \\
    |\delta| - \frac{1}{2} & otherwise.
    \end{cases}
$$ 

Инициализируем гиперпараметры нашей модели

In [ ]:
NUM_ENVS = 4
GAMMA=0.99
BATCH_SIZE=32
BUFFER_SIZE=1000000
MIN_REPLAY_SIZE=50000
EPSILON_END=0.1
EPSILON_DECAY=1000000
EXPLORATION_FRACTION = 0.5
TARGET_UPDATE_FREQ = 10000 // 4
LR = 5e-5
device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")

In [ ]:
dt = datetime.datetime.now().strftime("%Y-%m-%dT%H-%M-%S")
save_dir = "checkpoints/" + dt
logger = configure(save_dir, ["stdout", "csv", "tensorboard"])

Архитектура используемой нейронной сети:

  
        (0): Conv2d(3, 32, kernel_size=(8, 8), stride=(4, 4))

        (1): ReLU()

        (2): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2))

        (3): ReLU()

        (4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))

        (5): ReLU()
        
        (6): Flatten(start_dim=1, end_dim=-1)


Запустим обучение нашей нейросети

In [ ]:
model = DQN(env=envs, learning_rate=LR,buffer_size=BUFFER_SIZE, learning_starts=MIN_REPLAY_SIZE,\
            batch_size=BATCH_SIZE, policy="CnnPolicy", target_update_interval=TARGET_UPDATE_FREQ,\
            device=device, exploration_fraction=EXPLORATION_FRACTION, optimize_memory_usage=True,\
            exploration_final_eps=EPSILON_END, gamma=GAMMA)
model.set_logger(logger)

model.learn(total_timesteps=1000000, log_interval=4)
model.save(save_dir)

del model # remove to demonstrate saving and loading
load_dir = f"{save_dir}.zip"
# ld = "checkpoints/2022-07-03T01-32-50.zip"
model = DQN.load(load_dir)

Теперь посмотрим как играет наша модель

In [ ]:
env = make_atari_env("ALE/Breakout-v5", n_envs=1, vec_env_cls=DummyVecEnv, env_kwargs={"render_mode":"human"})
env = VecTransposeImage(env)
env = VecFrameStack(env, n_stack=4)
env = VecNormalize(env)
model = DQN.load("checkpoints/2022-07-03T01-32-50.zip")
obses = env.reset()
while True:
    actions, _states = model.predict(obses, deterministic=True)
    obses, rewards, dones, infos = env.step(actions)
    # env.render()

In [ ]:
%load_ext tensorboard
%tensorboard --logdir "checkpoints/2022-07-03T01-32-50"

<b>Вывод:</b> <i>обученная модель более менее неплохо играет, но т.к. вычислительные мощности не позволяют обучить до уровня серьезного ИИ, модель не показывает сильно впечатляющих результатов. 

Кода получилось не очень много, т.к. в основном использовались решения, доступные из коробки, но в рамках данного проекта я полностью разобрался как работает этот алгоритм, что находится под капотом у этой библиотеки. Тем не менее, эксперимент был проведен и были получены результаты.  

Дальнейшее поле для исследований <a href="https://arxiv.org/pdf/1710.02298.pdf">Rainbow: Combining Improvements in Deep Reinforcement Learning</a></i>